In [15]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

CODE TO LOAD MIN-MAX POOLED MFCC FILES that are stored in a particular folder

In [2]:
def load_mfcc_files(directory):
    mfcc_data = []
    for file in os.listdir(directory):
        if file.endswith('-MFCC.csv'):
            file_path = os.path.join(directory, file)
            data = pd.read_csv(file_path, header=None).values
            mfcc_data.append(data)
    return mfcc_data

code for scalling and building the cdbn model(subject to hyper-parameter tunin)

In [ ]:

def load_mfcc_files(directory):
    mfcc_data = []
    for file in os.listdir(directory):
        
            file_path = os.path.join(directory, file)
            data = pd.read_csv(file_path, header=None).values
            mfcc_data.append(data)
    return mfcc_data

# Load MFCC data
asha = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Asha Bhosle")
bhavgeet = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Bhavgeet")
kishor = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Kishor Kumar")
lavni = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Lavni")
michael = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Michael Jackson")
national = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\National Anthem")

# Debug: Print the number of files loaded from each directory
print(f"Asha Bhosle: {len(asha)} files")
print(f"Bhavgeet: {len(bhavgeet)} files")
print(f"Kishor Kumar: {len(kishor)} files")
print(f"Lavni: {len(lavni)} files")
print(f"Michael Jackson: {len(michael)} files")
print(f"National Anthem: {len(national)} files")

Asha Bhosle: 35 files
Bhavgeet: 29 files
Kishor Kumar: 44 files
Lavni: 25 files
Michael Jackson: 35 files
National Anthem: 29 files


In [9]:
def create_labeled_data(mfcc_data, label):
    labeled_data = []
    for data in mfcc_data:
        labeled_data.append([data, label])
    return labeled_data

# Create labeled data
asha_labeled = create_labeled_data(asha, 'Asha Bhosle')
bhavgeet_labeled = create_labeled_data(bhavgeet, 'Bhavgeet')
kishor_labeled = create_labeled_data(kishor, 'Kishor Kumar')
lavni_labeled = create_labeled_data(lavni, 'Lavni')
michael_labeled = create_labeled_data(michael, 'Michael Jackson')
national_labeled = create_labeled_data(national, 'National Anthem')

# Debug: Print the length of labeled data for each category
print(f"Asha Bhosle labeled: {len(asha_labeled)}")
print(f"Bhavgeet labeled: {len(bhavgeet_labeled)}")
print(f"Kishor Kumar labeled: {len(kishor_labeled)}")
print(f"Lavni labeled: {len(lavni_labeled)}")
print(f"Michael Jackson labeled: {len(michael_labeled)}")
print(f"National Anthem labeled: {len(national_labeled)}")

# Combine all labeled data
all_labeled_data = asha_labeled + bhavgeet_labeled + kishor_labeled + lavni_labeled + michael_labeled + national_labeled

# Convert to numpy array
all_labeled_data_array = np.array(all_labeled_data, dtype=object)

# Print the shape of the combined numpy array
print(all_labeled_data_array.shape)

Asha Bhosle labeled: 35
Bhavgeet labeled: 29
Kishor Kumar labeled: 44
Lavni labeled: 25
Michael Jackson labeled: 35
National Anthem labeled: 29
(197, 2)


In [25]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(all_labeled_data_array[:,1])

X=all_labeled_data_array[:,0]
X = X[..., np.newaxis]

x_train, x_test, y_train, y_test= train_test_split(X, y_encoded)
x_train.shape

(147, 1)

In [27]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.neural_network import BernoulliRBM

class RBMLayer(layers.Layer):
    def __init__(self, n_components, learning_rate=0.01, batch_size=10, n_iter=10, **kwargs):
        super(RBMLayer, self).__init__(**kwargs)
        self.rbm = BernoulliRBM(n_components=n_components, learning_rate=learning_rate, batch_size=batch_size, n_iter=n_iter)
        self.n_components = n_components
    
    def call(self, inputs):
        inputs_flat = tf.reshape(inputs, (tf.shape(inputs)[0], -1))
        transformed = self.rbm.transform(inputs_flat)
        return tf.reshape(transformed, (tf.shape(inputs)[0], self.n_components))
    
    def fit(self, X):
        X_flat = X.reshape((X.shape[0], -1))
        self.rbm.fit(X_flat)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.n_components)
    
from tensorflow.keras import models

def build_model(input_shape):
    """
    Build the specified model architecture.
    
    Parameters:
    input_shape (tuple): Shape of the input data.
    
    Returns:
    tensorflow.keras.Model: The model.
    """
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    
    # Conv1D Layer: 32 filters, filter size of 5, stride of 1
    model.add(layers.Conv1D(32, 5, activation='relu', strides=1))
    # Pooling Layer: Pool size of 2
    model.add(layers.MaxPooling1D(pool_size=2))
    
    # Conv1D Layer: 64 filters, filter size of 5, stride of 1
    model.add(layers.Conv1D(64, 5, activation='relu', strides=1))
    # Pooling Layer: Pool size of 2
    model.add(layers.MaxPooling1D(pool_size=2))
    
    model.add(layers.Flatten())
    
    # RBM Layer 1: 100 hidden units
    rbm_layer1 = RBMLayer(n_components=100)
    model.add(rbm_layer1)
    
    # RBM Layer 2: 50 hidden units
    rbm_layer2 = RBMLayer(n_components=50)
    model.add(rbm_layer2)
    
    # Fully Connected Layer: 128 units
    model.add(layers.Dense(128, activation='relu'))
    
    # Output Layer: Softmax with 6 units for each class
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

# Example usage
input_shape = (147, 1)  # Adjusted to match the input data shape
model = build_model(input_shape)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

code to visualize what cdbn is doing (subject to change)

In [10]:
def visualize_feature_maps(model, layer_name, input_data):
    """
    Visualize the feature maps of a convolutional layer.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the convolutional layer.
    input_data (numpy.ndarray): The input data.
    """
    layer = model.get_layer(name=layer_name)
    feature_map_model = models.Model(inputs=model.input, outputs=layer.output)
    feature_maps = feature_map_model.predict(input_data)
    
    n_features = feature_maps.shape[-1]
    fig, axes = plt.subplots(1, n_features, figsize=(20, 5))
    
    for i in range(n_features):
        axes[i].imshow(feature_maps[0, :, :, i], cmap='viridis')
        axes[i].axis('off')
    
    plt.show()

def visualize_filters(model, layer_name):
    """
    Visualize the filters of a convolutional layer.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the convolutional layer.
    """
    layer = model.get_layer(name=layer_name)
    filters, biases = layer.get_weights()
    
    # Normalize filter values to 0-1 so we can visualize them
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)
    
    n_filters = filters.shape[-1]
    fig, axes = plt.subplots(1, n_filters, figsize=(20, 5))
    
    for i in range(n_filters):
        f = filters[:, :, :, i]
        axes[i].imshow(f[:, :, 0], cmap='viridis')
        axes[i].axis('off')
    
    plt.show()

In [12]:

directory = "C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_processed" #path of the folder which contains the min-max pooled mfcc files
mfcc_predict_data = load_mfcc_files(directory)

mfcc_predict_data_combined = np.array(mfcc_predict_data)

# Print the shape of the combined numpy array
print(mfcc_predict_data_combined.shape)


(116, 20, 25000)


In [ ]:
mfccs_preprocessed = preprocess_mfcc(mfccs_data_combined)
input_shape = mfccs_preprocessed.shape[1:]
cdbn_model = build_cdbn(input_shape)
cdbn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


X_train = np.array([0])  # Replace with actual data
y_train = np.array([0])  # Replace with actual labels

# Train the model
cdbn_model.fit(X_train, y_train, epochs=10, batch_size=1)

visualize_filters(cdbn_model, 'conv2d')
visualize_feature_maps(cdbn_model, 'conv2d', mfccs_preprocessed)

# Predict on new data
X_test = np.array([0])  # Replace with actual test data
predictions = cdbn_model.predict(X_test)
print(predictions)